In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [10]:
df = np.load("Data/04_fv.np")
print("Size: ", df.size)
# print(df)

# Reverse the indexes and subtract all by 1 (Python is 0-indexed and MATLAB is 1-indexed)
# The features and channels dimensions are converted correctly
print(df[0, 0, :]) # [0.0024, 0.0012, 0.0014] (all features, channel 1, trial 1)
print(df[0, 1, :]) # [0.0015, 0.0007, -0.0008] (all features, channel 2, trial 1)
print(df[0, 2, :]) # [0.0052, 0.0032, -0.0017] (all features, channel 3, trial 1)
print(df[0, 26, :]) # [-.0008897, -0.5272, -0.7639] (all features, channel 27, trial 1)

# For some reason, I believe the trials are out of order
print(df[1, 0, :]) # [0.0016, 0.0023, 0.0038] INCORRECT for trial 2
print(df[2, 0, :]) # [-0.0013, -0.0023, -0.0047] INCORRECT for trial 3

Size:  9000
[ 0.00241223  0.0011588  -0.00137351]
[ 0.00145174  0.00065382 -0.00076341]
[ 0.00519325  0.00315593 -0.00166335]
[-0.00088966 -0.0005272  -0.00076389]
[0.00164122 0.00226265 0.00379152]
[-0.00133688 -0.00231209 -0.00466555]


In [11]:
def distance(slice1, slice2):
    """
    Calculate the Euclidean distance between two 3D slices (40, 3) from a (75, 40, 3) array.
    """
    return np.sqrt(np.sum((slice1 - slice2)**2))

def kmeans(df, k=4, tol=0.05):
    """
    K-means clustering for 3D data.

    Parameters:
        df (numpy.ndarray): 3D numpy array with shape (75, 40, 3), each slice (40, 3) treated as a point.
        k (int): Number of clusters.
        tol (float): Tolerance for L_2 convergence check on centroids.

    Returns:
        centroids (numpy.ndarray): Array of centroids, one for each cluster.
        clusters (numpy.ndarray): Cluster assignment for each point.
        rec_error (float): Reconstruction error on final iteration.
    """    
    # Initialize reconstruction error for 1st iteration
    prev_rec_error = np.inf
    
    # Random centroids from data
    clocs = np.random.choice(df.shape[0], size=k, replace=False)
    centroids = df[clocs, :, :].copy()
    
    # Initialize objects for points-cluster distances and cluster assignments.
    dists = np.zeros((k, df.shape[0]))
    clusters = np.array([-1] * df.shape[0])
    
    # Index and convergence trackers
    ii = 0
    Done = False
    while not Done:
        # Update classifications
        for ji in range(k):
            for pi in range(df.shape[0]):
                dists[ji, pi] = distance(df[pi, :, :], centroids[ji, :, :])
        
        clusters = dists.argmin(axis=0)
        
        # Update centroids
        for ji in range(k):
            if np.sum(clusters == ji) > 0:
                centroids[ji, :, :] = np.mean(df[clusters == ji], axis=0)
            else:
                # Reinitialize centroid if no points are assigned to prevent empty clusters
                centroids[ji, :, :] = df[np.random.choice(df.shape[0], size=1), :, :]

        # Calculate Reconstruction Error    
        rec_error = np.sum(np.min(dists, axis=0)**2) / df.shape[0]
        
        # Convergence check
        change_in_error = np.abs(prev_rec_error - rec_error)
        if change_in_error < tol:
            print(f'Done at iteration {ii} with change of {change_in_error}')
            Done = True
        elif ii == 50:
            print('No convergence in 50 steps')
            Done = True
        
        prev_rec_error = rec_error
        ii += 1
    
    return centroids, clusters, rec_error

In [12]:
centroids, clusters, meanerror=kmeans(df, k=3)
print('Final mean error:', meanerror)
print(clusters)
# plt.scatter(df[0:75], df[0:75],c=clusters)

Done at iteration 1 with change of 0.0005295649236366534
Final mean error: 0.0015411218048231256
[1 2 3 1 1 2 1 2 1 2 2 3 3 1 1 1 3 2 3 1 1 3 3 3 2 1 1 3 1 2 3 2 2 2 1 3 1
 1 2 1 2 1 1 3 1 1 0 1 1 2 1 1 1 3 2 1 2 1 1 3 1 1 1 2 3 1 1 1 1 1 3 2 2 1
 3]


In [13]:
# Loading ground truth
import scipy.io as sp
ground_truth_struct = sp.loadmat('Data/04_labels.mat')
# print(ground_truth_struct['Y'])

gt = []
for elem in ground_truth_struct['Y']:
    gt.append(elem[0])

gt_np = np.array(gt)
print("Ground Truth: ", gt_np)

correct = 0
for i in range(len(gt_np)):
    if gt_np[i] == clusters[i]:
        correct += 1
print("Clustering Accuracy: ", correct/len(gt_np))

Ground Truth:  [1 2 3 1 3 2 2 3 1 1 2 3 3 2 1 2 3 1 2 2 1 3 3 2 1 2 3 1 2 1 3 3 1 2 2 3 1
 1 3 2 3 1 2 3 3 2 1 1 3 2 3 1 2 3 2 1 1 2 3 3 1 2 2 1 3 1 2 3 1 1 3 2 2 1
 3]
Clustering Accuracy:  0.5066666666666667
